In [15]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot

init_notebook_mode(connected=True)

import plotly.plotly as py
import plotly.graph_objs as go

import igraph
from igraph import *
igraph.__version__

'0.7.1'

In [16]:
# plotly.__version__

In [17]:
def get_layer(i):
    return int(math.floor(math.log(float(i) + 1, 2)))

def get_parent(i):
    return int(math.floor((i - 1) /2))

In [18]:
def get_parents(i, no_root=True):
    res = []
    while True:
        i = get_parent(i)
        if i > 0:
            res.append(i)
        else:
            break
    return res

In [19]:
def create_sequence(i):
    return '<thr> ' + ' '.join(['<com> ' + str(n) + ' </com>' for n in get_parents(i) + [i]]) + ' </thr>'

In [24]:
def tree_plot(nr_vertices = 25, highlight = None):
    v_label = map(str, range(nr_vertices))
    G = Graph.Tree(nr_vertices, 2) # 2 stands for children number
    lay = G.layout('rt', root=[0])

    position = {k: lay[k] for k in range(nr_vertices)}
    Y = [lay[k][1] for k in range(nr_vertices)]
    M = max(Y)

    es = EdgeSeq(G) # sequence of edges
    E = [e.tuple for e in G.es] # list of edges

    L = len(position)
    Xn = [position[k][0] for k in range(L)]
    Yn = [2*M-position[k][1] for k in range(L)]
    Xe = []
    Ye = []
    for edge in E:
        Xe+=[position[edge[0]][0],position[edge[1]][0], None]
        Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None] 

    labels = v_label

    labels = ['article']
    for l in v_label[1:]:
        labels.append(l + ',' + str(get_layer(l)))

    colors = ['#989898']
    if not highlight is None:
        parents_highlight = get_parents(highlight)

        for i in range(1, nr_vertices):
            if i in parents_highlight:
                colors.append('#EE7600')
            else:
                if i == highlight:
                    colors.append('#DB4551')
                else:
                    colors.append('#6175c1')
    else:
        colors += ['#6175c1'] * (nr_vertices - 1)

    lines = go.Scatter(x=Xe,
                       y=Ye,
                       mode='lines',
                       line=dict(color='rgb(210,210,210)', width=1),
                       hoverinfo='none'
                       )
    dots = go.Scatter(x=Xn,
                      y=Yn,
                      mode='markers',
                      name='',
                      marker=dict(symbol='dot',
                                  size=35,
                                  color=colors,
                                  line=dict(color='rgb(50,50,50)', width=1)
                                  ),
                      text=labels,
                      hoverinfo='text',
                      opacity=0.8
                      )

    def make_annotations(pos, text, font_size=10, font_color='rgb(250,250,250)'):
        L=len((pos))
        if len((text))!=L:
            raise ValueError('The lists pos and text must have the same len')
        annotations = go.Annotations()
        for k in range(L):
            annotations.append(
                go.Annotation(
                    text=labels[k], # or replace labels with a different list for the text within the circle  
                    x=pos[k][0], y=2*M-position[k][1],
                    xref='x1', yref='y1',
                    font=dict(color=font_color, size=font_size),
                    showarrow=False)
            )
        return annotations  

    axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
                zeroline=False,
                showgrid=False,
                showticklabels=False,
                )
    title = ''
    if not highlight is None:
        title=  create_sequence(highlight)
    layout = dict(title=title,  
                  annotations=make_annotations(position, v_label),
                  font=dict(size=12),
                  showlegend=False,
                  xaxis=go.XAxis(axis),
                  yaxis=go.YAxis(axis),          
                  margin=dict(l=40, r=40, b=85, t=100),
                  hovermode='closest',
                  plot_bgcolor='rgb(248,248,248)'          
                  )

    data=go.Data([lines, dots])
    fig=dict(data=data, layout=layout)
    fig['layout'].update(annotations=make_annotations(position, v_label))
    return iplot(fig, filename='Tree-Reingold-Tilf')

In [25]:
for i in range(1, 20):
    tree_plot(25, i)